In [5]:
using DynamicPolynomials, LinearAlgebra, SparseArrays

println("***Problem setting***")

n=1000

println("Number of variable: n=",n)
println("====================")

@polyvar x[1:n]# variables

#function to get a random quadratic polynomial of variables x(T)
function generate_random_poly(T::UnitRange{Int64})
    v=reverse(monomials(x[T],0:2))
    c=2*rand(Float64,length(v)).-1
    return c'*v
end

# ball constraints on subsets of variables
u=10# clique size
p=floor(Int64,n/u) #number of cliques

# indices of variables on each clique
I=Vector{UnitRange{Int64}}(undef,p)
I[1]=1:u 
I[2:p-1]=[u*(j-1):u*j for j in 2:p-1]
I[p]=u*(p-1):n

# random quadratic objective function f
vecf=[generate_random_poly(I[j]) for j in 1:p] #vector of separable polynomials on each clique
f=sum(vecf)

# ball constraints on each clique
m=ceil(Int64, n/5)# number of the equality constraints

r=floor(Int64,m/p)
J=[(j-1)*r+1:j*r for j in 1:p-1]# assign equality constraints
append!(J,[(p-1)*r+1:m])

g=Vector{Polynomial{true,Float64}}(undef,m)


# get a random point satisfies the inequality constraints
randx=rand(Float64,n)

for j in 1:p
    randx[I[j]]=randx[I[j]]./sum(randx[I[j]])
    randx[I[j]]=rand(Float64,1)[1]*randx[I[j]]
end

for j in 1:p
    for i in J[j]
        g[i]=generate_random_poly(I[j])
        g[i]+=0.125-g[i](x => randx) #make the random point satisfy the equality constraint h[i](randx) = 0
    end
    g[J[j][1]]=1-sum(x[I[j]])
end

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")


l=ceil(Int64, n/5)# number of the equality constraints

r=floor(Int64,l/p)
W=[(j-1)*r+1:j*r for j in 1:p-1]# assign equality constraints
append!(W,[(p-1)*r+1:l])

h=Vector{Polynomial{true,Float64}}(undef,l)


for j in 1:p
    for i in W[j]
        h[i]=generate_random_poly(I[j])
        h[i]-=h[i](x => randx) #make the random point satisfy the equality constraint h[i](randx) = 0
    end
end

l=length(h)
println("Number of equality constraints: l=",l)


***Problem setting***
Number of variable: n=1000
Number of inequality constraints: m=200
Number of equality constraints: l=200


In [6]:
using SparseArrays

include("../src/InterruptedRelax.jl")
using .InterruptedRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterruptedRelax.get_info(x,f,g,h,sparse=true);

#InterruptedRelax.save_info_sparsePOP(randx,u,n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh)

#include("./sparsePOP3.jl")

lmon_g=[lmon_g;ones(UInt64,n)]
alpha=spzeros(UInt64,n,1)

for j in 1:n
    alpha=spzeros(UInt64,n,1)
    alpha[j]=1
    supp_g=[supp_g;[alpha]]
    coe_g=[coe_g;[ones(Float64,1)]]

    dg=[dg;1]
end

using TSSOS

k=2

println("Maximal matrix size: ",binomial(u+1+k,k))

@time opt,sol,data=TSSOS.cs_tssos_first(Vector{SparseMatrixCSC{UInt8,UInt32}}([[supp_f];supp_g;supp_h]),[[coe_f];coe_g;coe_h],n,k,[dg;dh],numeq=l,CS="MD",TS=false);

Maximal matrix size: 78
------------------------------------------------------
The clique sizes of varibles:
[11, 10]
[99, 1]
------------------------------------------------------
Problem


  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 135641          
  Cones                  : 0               
  Scalar variables       : 15577           
  Matrix variables       : 1300            
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.03            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.16    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization prob

In [13]:
using SparseArrays

include("../src/InterruptedRelax.jl")
using .InterruptedRelax


n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterruptedRelax.get_info(x,f,g,h,sparse=true);

#InterruptedRelax.save_info_sparsePOP(randx,u,n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh)

#include("./sparsePOP3.jl")

d=Int64(maximum([sum(supp_f[:,i]) for i in 1:lmon_f]))

println("Degree: d=",d)


k=1

println("Relaxation order: k=",k)


s=12


println("Block size: s=",s)

@time opt_val1=InterruptedRelax.RelaxSparse(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,k,s,d;assign="min",alg="MD",minimize=true)

Degree: d=2
Relaxation order: k=1
Block size: s=12
------------------------------------------------------


The clique sizes of varibles:
[11, 10]
[99, 1]
------------------------------------------------------
  Number of cliques: p=100
  Largest clique size: u=11
Maximal matrix size:12
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 43813           
  Cones                  : 0               
  Scalar variables       : 39920           
  Matrix variables       : 1299            
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 198
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time            

-63.42506128755418